# Compute per-hex depth quantile from EMODnet Bathymetry 2024

Replaces the mean/median depth in `meta.json` with a low quantile (10th percentile)
of bathymetry sampled within each hex polygon.

**Why:** mean/median depth misclassifies hexes near steep coasts (e.g. Norway) as
non-habitable even when a substantial fraction of their area is shallower than 85 m.
The NWS model bathy only covers −16°W to 13°E and up to 62.7°N; EMODnet 2024 covers
the full NW European shelf.

**Source:** `bathymetry_dtm_2024` from the EMODnet ERDDAP at
`https://erddap.emodnet.eu/erddap/` (~115 m resolution). The full grid is ~130 GB;
we fetch only per-hex bounding box subsets via OPeNDAP — no credentials required.

**Output:** updated `database/data/meta.json` with `depth` replaced by the 10th
percentile of ocean depth within each hex.

In [ ]:
import json
import numpy as np
import xarray as xr
import geopandas as gpd
import shapely
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
# --- parameters ---
DEPTH_QUANTILE = 0.10   # 10th percentile

# EMODnet Bathymetry 2024 via ERDDAP OPeNDAP — no auth required
# Full grid is ~130 GB; we subset per-hex via OPeNDAP so only bbox windows are downloaded
ERDDAP_BASE = "https://erddap.emodnet.eu/erddap/griddap"
DATASET_ID  = "bathymetry_dtm_2024"
OPENDAP_URL = f"{ERDDAP_BASE}/{DATASET_ID}"

# When run via papermill, cwd is preproc/; when run interactively, cwd is preproc/notebooks/
_cwd = Path.cwd()
if (_cwd / "../database/data").exists():
    OUT_DIR = (_cwd / "../database/data").resolve()
else:
    OUT_DIR = (_cwd / "../../database/data").resolve()

META_PATH = OUT_DIR / "meta.json"
print(f"OUT_DIR: {OUT_DIR}")

In [ ]:
# Open and download the regional subset from EMODnet ERDDAP (~2 GB, ~3 min on 100 Mbit/s)
# EMODnet elevation: negative = ocean, positive/zero = land
LON_MIN, LON_MAX = -30.0, 13.0
LAT_MIN, LAT_MAX =  46.0, 66.0

print("Opening EMODnet ERDDAP via OPeNDAP ...")
ds = xr.open_dataset(OPENDAP_URL, engine="pydap")
print(ds)
print("\nSubsetting and downloading region ...")
elev = (
    ds["elevation"]
    .sel(latitude=slice(LAT_MIN, LAT_MAX), longitude=slice(LON_MIN, LON_MAX))
    .load()
)
print(f"Downloaded: {elev.shape}, {elev.nbytes / 1e9:.2f} GB")
print(f"Elevation range: {float(elev.min()):.0f} – {float(elev.max()):.0f} m")

In [ ]:
# Land mask: elevation < 0 → ocean; depth = -elevation
ocean_mask = elev < 0
bathy_ocean = (-elev).where(ocean_mask)   # positive depth on ocean, NaN on land

print(f"Ocean cells:    {int(ocean_mask.sum())}")
print(f"Land/dry cells: {int((~ocean_mask).sum())}")
print(f"Depth range (ocean only): {float(bathy_ocean.min()):.0f} – {float(bathy_ocean.max()):.0f} m")

In [ ]:
# Load hex polygons
hexes = gpd.read_file(OUT_DIR / "hexes.geojson")
print(f"Hexes: {len(hexes)}")

In [ ]:
def hex_depth_quantile(geom, q=DEPTH_QUANTILE):
    """Return q-th quantile of ocean depth (m) within hex polygon.
    Uses bathy_ocean (land already NaN-masked).
    """
    minx, miny, maxx, maxy = geom.bounds
    local = bathy_ocean.sel(
        latitude=slice(miny - 0.01, maxy + 0.01),
        longitude=slice(minx - 0.01, maxx + 0.01),
    )
    if local.size == 0:
        return np.nan
    lons = local.longitude.values
    lats = local.latitude.values
    LON_G, LAT_G = np.meshgrid(lons, lats)
    inside = shapely.contains_xy(geom, LON_G.ravel(), LAT_G.ravel())
    vals = local.values.ravel()[inside]
    ocean = vals[np.isfinite(vals)]
    if len(ocean) == 0:
        return np.nan
    return float(np.percentile(ocean, q * 100))

print(hex_depth_quantile(hexes.geometry.iloc[0]))

In [ ]:
depth_q = np.full(len(hexes), np.nan)

for i, geom in enumerate(tqdm(hexes.geometry, desc="hex depth q10")):
    depth_q[i] = hex_depth_quantile(geom)

hexes["depth_q10"] = depth_q
print(f"Done. NaN count: {np.isnan(depth_q).sum()} / {len(depth_q)}")
print(f"Depth q10 range: {np.nanmin(depth_q):.0f} – {np.nanmax(depth_q):.0f} m")

In [ ]:
import matplotlib.pyplot as plt

meta = json.loads(META_PATH.read_text())
old_depth = np.array([meta["depth"].get(str(i), np.nan) for i in hexes["id"]])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, vals, title in zip(
    axes,
    [old_depth, depth_q],
    ["Old: depth_median (from connectivity NetCDF)", f"New: depth_q{int(DEPTH_QUANTILE*100)} (NWS model bathy)"],
):
    sc = ax.scatter(
        hexes.geometry.centroid.x, hexes.geometry.centroid.y,
        c=vals, cmap="Blues", vmin=0, vmax=200, s=2,
    )
    ax.set_title(title)
    plt.colorbar(sc, ax=ax, label="depth (m)")
plt.tight_layout()
plt.savefig(OUT_DIR / "depth_comparison.png", dpi=150)
plt.show()

In [ ]:
# Show change in habitable classification
was_nonhabitable = old_depth > 85
now_nonhabitable = depth_q  > 85
newly_habitable  = was_nonhabitable & ~now_nonhabitable
print(f"Previously non-habitable (depth_median > 85 m): {was_nonhabitable.sum()}")
print(f"Now non-habitable (depth_q10 > 85 m):           {now_nonhabitable.sum()}")
print(f"Newly classified as habitable:                   {newly_habitable.sum()}")

In [ ]:
# Write updated meta.json
meta["depth"] = {
    str(int(row["id"])): (None if np.isnan(row["depth_q10"]) else row["depth_q10"])
    for _, row in hexes.iterrows()
}

META_PATH.write_text(json.dumps(meta))
print(f"Written: {META_PATH}")